# Dask Delayed

Материалы:
* Макрушин С.В. Лекция 13: Dask Delayed
* https://docs.dask.org/en/latest/delayed.html
* JESSE C. DANIEL. Data Science with Python and Dask.


## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

## Лабораторная работа 14

1. Напишите функцию, которая считывает файл формата xml из архива `reviewers_full.zip` и по данным этого файла формирует список словарей, содержащих следующие ключи: `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [2]:
import dask.delayed

In [3]:
%%file multi_util.py
from bs4 import BeautifulSoup


def get_reviewers_dict(filename):
    with open(filename, "r") as fp:
        data = BeautifulSoup(fp, 'lxml')
        reviewers = []

        for el in data.find_all("user"):
            id = int(exists(el.find("id")))
            username = exists(el.find("username"))
            name = exists(el.find("name"))
            sex = exists(el.find("sex"))
            country = exists(el.find("country"))
            mail = exists(el.find("mail"))
            registered = exists(el.find("registered"))
            birthdate = exists(el.find("birthdate"))
            name_prefix = el.get("prefix")

            if el.find("country") is None:
                country_code = None
            else:
                country_code = el.find("country").get("code")

            user = {"id": id, "username": username, "name": name, "sex": sex,
                    "country": country, "mail": mail, "registered": registered,
                    "birthdate": birthdate, "name_prefix": name_prefix,
                    "country_code": country_code}

            reviewers.append(user)
        return reviewers


def exists(value):
    return value if value is None else value.text


Overwriting multi_util.py


In [4]:
import multi_util
from multiprocessing import Pool

def task_multi():
    filenames = [f"./data/reviewers_full/reviewers_full_{i}.xml" for i in range(5)]
    with Pool(5) as pool:
        result_list = pool.map(multi_util.get_reviewers_dict, filenames)
    return result_list

def task_multi_delayed():
    filenames = [f"./data/reviewers_full/reviewers_full_{i}.xml" for i in range(5)]
    with Pool(5) as pool:
        result_list = pool.map(dask.delayed(multi_util.get_reviewers_dict), filenames)
    return result_list

2. Измерьте время выполнения функции из задания 1 на всех файлах из архива. Ускорьте время выполнения, используя `dask.delayed`.

In [5]:
%%time
result_pool = task_multi()

CPU times: user 334 ms, sys: 126 ms, total: 460 ms
Wall time: 26.4 s


In [ ]:
result_delayed = task_multi_delayed()

In [ ]:
%%time
result_delayed_mp = dask.compute(result_delayed, scheduler='multiprocessing')

CPU times: user 194 ms, sys: 61.1 ms, total: 255 ms
Wall time: 1min 19s


In [ ]:
%%time
result_delayed_thread = dask.compute(result_delayed, scheduler='threading')

CPU times: user 1min 53s, sys: 37.8 s, total: 2min 31s
Wall time: 2min


3. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [6]:
import dask.bag as db

In [7]:
reviewers = task_multi_delayed()

In [12]:
reviewers_bag = db.from_delayed(reviewers)
reviewers_bag

dask.bag<bag-from-delayed, npartitions=5>

In [9]:
def birth_year(data):
    if data["birthdate"] is not None:
        year = int(data["birthdate"].split("-")[0])
        if year < 1980:
            data["birth_year"] = year
            return data


In [13]:
reviewers_bag = reviewers_bag.filter(birth_year)

4. Из `dask.bag`, полученного в задании 3, создайте `dask.dataframe` при помощи метода `bag.to_dataframe`. Укажите столбец `id` в качестве индекса.

In [15]:
reviewers_df = reviewers_bag.to_dataframe().set_index("id")

In [16]:
reviewers_df.head()

,username,name,sex,country,mail,registered,birthdate,name_prefix,country_code,birth_year
id,,,,,,,,,,
1533,joseph38,None,None,None,josephroberts@gmail.com,None,1968-03-20,None,None,1968
1773,millercory,None,None,Yemen,None,None,1965-04-24,None,YE,1965
1962,joshuamunoz,Bobby Cohen,M,None,xduncan@gmail.com,2005-05-28,1965-02-22,Dr.,None,1965
2008,pphillips,Nicole Johnson,None,Bosnia and Herzegovina,ylam@gmail.com,None,1958-02-05,None,BA,1958
2054,barnesdoris,Carl Ingram,None,Benin,None,2017-05-30,1974-02-22,None,BJ,1974


5. Назовем отзыв негативным, если оценка равна 0, 1 или 2. Загрузите данные о негативных отзывах из файлов архива `reviews_full` (__ЛР12__) в виде `dask.DataFrame`. Посчитайте количество отзывов с группировкой по пользователю, оставившему отзыв. Объедините результат с таблицей, полученной в задаче 4.

In [17]:
import json

In [18]:
reviews_full = db.read_text("./data/reviews_full/reviews_*.json").map(json.loads).to_dataframe()

In [20]:
reviews_count = reviews_full.groupby(reviews_full["user_id"])["review"].count().compute()

In [21]:
reviewers_df.join(reviews_count.to_frame(), on="id").head()

,username,name,sex,country,mail,registered,birthdate,name_prefix,country_code,birth_year,review
id,,,,,,,,,,,
1533,joseph38,None,None,None,josephroberts@gmail.com,None,1968-03-20,None,None,1968,1024
1773,millercory,None,None,Yemen,None,None,1965-04-24,None,YE,1965,29
1962,joshuamunoz,Bobby Cohen,M,None,xduncan@gmail.com,2005-05-28,1965-02-22,Dr.,None,1965,148
2008,pphillips,Nicole Johnson,None,Bosnia and Herzegovina,ylam@gmail.com,None,1958-02-05,None,BA,1958,18
2054,barnesdoris,Carl Ingram,None,Benin,None,2017-05-30,1974-02-22,None,BJ,1974,7
